<a href="https://colab.research.google.com/github/Rex-Chan99/Bitcoin_prediction/blob/main/Master_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas_ta
!pip install numpy==1.23.5
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218910 sha256=ac20c6d49f9ad3151a9480528ff62e6354b6faff51d84928233da586efba13ae
  Stored in directory: /root/.cache/pip/wheels/7f/33/8b/50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas_ta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 51.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
pymc 5.21.2 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
scikit-im

In [32]:
!git clone https://github.com/ArthurChan-1111/Bitcoin_prediction.git

%cd Bitcoin_prediction

import pandas as pd
import pandas_ta as ta
import csv
import numpy as np

btc_data = pd.read_csv('Bitcoin Historical Data.csv', on_bad_lines='skip', lineterminator='\n')

Cloning into 'Bitcoin_prediction'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 79 (delta 24), reused 13 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (79/79), 3.95 MiB | 16.35 MiB/s, done.
Resolving deltas: 100% (24/24), done.
/content/Bitcoin_prediction/Bitcoin_prediction/Bitcoin_prediction/Bitcoin_prediction/Bitcoin_prediction


In [33]:
# Convert the dictionary to a DataFrame
btc_data = pd.DataFrame(btc_data)

# 1. Parse the "Date" column into datetime format
btc_data["Date"] = pd.to_datetime(btc_data["Date"], format="%m/%d/%Y")

# 2. Sort the data by date in ascending order
# 2. Set the "Date" column as the index
btc_data.set_index("Date", inplace=True)
btc_data.sort_values(by="Date", ascending=True, inplace=True)

#3. rename to Vol
btc_data.rename(columns={"Vol. ('000)": "Vol"}, inplace=True)
btc_data.rename(columns={"Change %\r": "Change"}, inplace=True)

# 4. Remove '%' from "Change %" and convert to numeric
btc_data["Change"] = btc_data["Change"].str.replace("%", "").str.strip().astype(float)

# 5. Convert numeric columns to proper types
btc_data["Price"] = pd.to_numeric(btc_data["Price"], errors="coerce")
btc_data["Open"] = pd.to_numeric(btc_data["Open"], errors="coerce")
btc_data["High"] = pd.to_numeric(btc_data["High"], errors="coerce")
btc_data["Low"] = pd.to_numeric(btc_data["Low"], errors="coerce")
btc_data["Vol"] = pd.to_numeric(btc_data["Vol"], errors="coerce")

btc_data.tail(5)

,Price,Open,High,Low,Vol,Change
Date,,,,,,
2025-04-01,85164.2,82548.6,85506.2,82435.8,66.72,3.17
2025-04-02,82525.0,85166.0,88377.3,82358.9,104.05,-3.10
2025-04-03,83159.4,82522.6,83908.9,81313.8,83.15,0.77
2025-04-04,83866.7,83160.2,84704.1,81691.1,91.54,0.85
2025-04-05,83552.1,83876.0,84249.3,82384.4,32.71,-0.38


In [41]:
# Calculate RSI, EMA, SMA, and MACD
btc_data["RSI"] = ta.rsi(btc_data["Price"], length=14)  # Relative Strength Index
btc_data["EMA"] = ta.ema(btc_data["Price"], length=14)  # Exponential Moving Average
btc_data["SMA"] = ta.sma(btc_data["Price"], length=14)  # Simple Moving Average

# Calculate MACD --------------------------------------------------------------------------------------
macd = ta.macd(btc_data["Price"], fast=12, slow=26, signal=9)
btc_data["MACD"] = macd["MACD_12_26_9"]
btc_data["MACD_Signal"] = macd["MACDs_12_26_9"]
btc_data["MACD_Hist"] = macd["MACDh_12_26_9"]

# Categorical MACD - Buy (1) and Sell (0) signals
btc_data["C_MACD"] = None  # Default to None (no signal)
btc_data.loc[
    (btc_data["MACD"] > btc_data["MACD_Signal"]) &
    (btc_data["MACD"].shift(1) <= btc_data["MACD_Signal"].shift(1)),
    "C_MACD"] = 1  # Buy signal # Detect Buy Signal (Crossover Up)
btc_data.loc[
    (btc_data["MACD"] <= btc_data["MACD_Signal"]) &
    (btc_data["MACD"].shift(1) > btc_data["MACD_Signal"].shift(1)),
    "C_MACD"] = 0  # Sell signal # Detect Sell Signal (Crossover Down)

# Calculate Bollinger Bands --------------------------------------------------------------------------------------
bbands = ta.bbands(btc_data["Price"], length=20, std=2)
btc_data["BB_Lower"] = bbands["BBL_20_2.0"]  # Lower Band
btc_data["BB_Upper"] = bbands["BBU_20_2.0"]  # Upper Band
btc_data["BB_Middle"] = bbands["BBM_20_2.0"]  # Middle Band

# Initialize C_BB column (default to no signal)
btc_data["C_BB"] = None

# Generate Buy and Sell Signals based on Bollinger Band crossovers
btc_data.loc[
    (btc_data["Price"] < btc_data["BB_Lower"]) & (btc_data["Price"].shift(1) >= btc_data["BB_Lower"].shift(1)),
    "C_BB"] = 1  # Buy signal (Price crosses below Lower Band)

btc_data.loc[
    (btc_data["Price"] > btc_data["BB_Upper"]) & (btc_data["Price"].shift(1) <= btc_data["BB_Upper"].shift(1)),
    "C_BB"] = 0  # Sell signal (Price crosses above Upper Band)

# Calculate ATR --------------------------------------------------------------------------------------
btc_data["ATR"] = ta.atr(btc_data["High"], btc_data["Low"], btc_data["Price"], length=14)

# Set Stop-Loss Levels (Example with Long Trade)
atr_multiplier = 2
btc_data["Stop_Loss_Long"] = btc_data["Price"] - (btc_data["ATR"] * atr_multiplier) #Stop-Loss for Buy (Long) Trade
btc_data["Stop_Loss_Short"] = btc_data["Price"] + (btc_data["ATR"] * atr_multiplier) #Stop-Loss for Sell (Short) Trade

# Calculate VWAP --------------------------------------------------------------------------------------
btc_data["VWAP"] = ta.vwap(btc_data["High"], btc_data["Low"], btc_data["Price"], btc_data["Vol"])

# Initialize the "C_VWAP" column
btc_data["C_VWAP"] = None
# Generate buy and sell signals based on VWAP crossover
btc_data.loc[
    (btc_data["Price"] > btc_data["VWAP"]) & (btc_data["Price"].shift(1) <= btc_data["VWAP"].shift(1)),
    "C_VWAP"] = 1  # Buy signal (Price crosses above VWAP)

btc_data.loc[
    (btc_data["Price"] < btc_data["VWAP"]) & (btc_data["Price"].shift(1) >= btc_data["VWAP"].shift(1)),
    "C_VWAP"] = 0  # Sell signal (Price crosses below VWAP)

# Display the first few rows of the data with calculated indicators
btc_data.tail(50)

,Price,Open,High,Low,Vol,Change,RSI,EMA,SMA,MACD,...,C_MACD,BB_Lower,BB_Upper,BB_Middle,C_BB,ATR,Stop_Loss_Long,Stop_Loss_Short,VWAP,C_VWAP
Date,,,,,,,,,,,,,,,,,,,,,
2025-02-15,97573.4,97504.4,97973.2,97210.8,29.65,0.07,46.995267,97862.179958,97299.000000,-917.924487,...,None,93421.329831,104293.480169,98857.405,None,3544.102262,90485.195477,104661.604523,97585.800000,None
2025-02-16,96132.1,97573.1,97706.5,96060.2,33.35,-1.48,43.392847,97631.502630,97188.085714,-972.805206,...,None,93212.550253,103906.299747,98559.425,None,3408.544957,89315.010086,102949.189914,96632.933333,None
2025-02-17,95781.3,96120.2,97002.3,95222.0,58.40,-0.36,42.538156,97384.808946,96791.242857,-1032.700867,...,None,92964.451055,103597.598945,98281.025,None,3292.241746,89196.816508,102365.783492,96001.866667,None
2025-02-18,95638.0,95781.5,96645.1,93403.4,74.09,-0.15,42.172735,97151.901087,96637.285714,-1079.290335,...,None,93071.752431,102681.897569,97876.825,None,3288.631621,89060.736758,102215.263242,95228.833333,1
2025-02-19,96644.8,95663.7,96829.6,95049.1,58.27,1.05,45.701972,97084.287609,96640.821429,-1023.177920,...,None,93818.354034,101127.085966,97472.720,None,3180.907934,90282.984132,103006.615868,96174.500000,None
2025-02-20,98310.3,96645.2,98721.6,96441.9,61.84,1.72,51.026659,97247.755928,96765.214286,-834.694656,...,1,94364.952277,100169.097723,97267.025,None,3116.535939,92077.228123,104543.371877,97824.600000,None
2025-02-21,96155.6,98310.3,99473.4,94903.6,104.28,-2.19,44.893051,97102.135137,96739.650000,-849.395293,...,None,94556.230382,99528.579618,97042.405,None,3220.340514,89714.918971,102596.281029,96844.200000,0
2025-02-22,96556.1,96158.3,96930.8,95801.1,45.62,0.42,46.187842,97029.330452,96747.378571,-819.284450,...,None,94509.463257,99462.466743,96985.965,None,3071.009049,90414.081902,102698.118098,96429.333333,1
2025-02-23,96258.8,96556.2,96655.2,95259.5,38.82,-0.31,45.336280,96926.593059,96732.350000,-810.072998,...,None,95250.682996,98213.417004,96732.050,None,2951.344117,90356.111766,102161.488234,96057.833333,None


In [ ]:
# Calculate Bollinger Bands
bollinger_bands = ta.bbands(btc_data["Price"], length=20, std=2)
btc_data["BB_Upper"] = bollinger_bands["BBU_20_2.0"]
btc_data["BB_Middle"] = bollinger_bands["BBM_20_2.0"]
btc_data["BB_Lower"] = bollinger_bands["BBL_20_2.0"]

# Percentage difference between Bollinger Bands
btc_data["BB_Percentage"] = (
    (btc_data["BB_Upper"] - btc_data["BB_Lower"]) / btc_data["BB_Middle"])

In [ ]:
# Function to clean and convert volume data
def clean_volume(volume):
    if isinstance(volume, str):  # Check if the value is a string
        volume = volume.replace(',', '')  # Remove commas
        if 'B' in volume:  # If the value contains 'B' (billions)
            return float(volume.replace('B', '')) * 1_000_000_000
        elif 'M' in volume:  # If the value contains 'M' (millions)
            return float(volume.replace('M', '')) * 1_000_000
        elif 'K' in volume:  # If the value contains 'K' (thousands)
            return float(volume.replace('K', '')) * 1_000
        else:  # If no suffix is present, convert to float directly
            return float(volume)
    return np.nan  # Handle unexpected cases

# Apply the cleaning function to the volume column
btc_data["Volume"] = btc_data["Vol. ('000)"].apply(clean_volume)

In [ ]:
# Add On-Balance Volume (OBV)
btc_data["OBV"] = ta.obv(btc_data["Price"], btc_data["Volume"])

In [ ]:
# Day of the week (0=Monday, 6=Sunday)
btc_data["Day_of_Week"] = btc_data["Date"].dt.dayofweek

# Week of the year
btc_data["Week_of_Year"] = btc_data["Date"].dt.isocalendar().week

# Month of the year
btc_data["Month"] = btc_data["Date"].dt.month

# Quarter of the year
btc_data["Quarter"] = btc_data["Date"].dt.quarter

# Year
btc_data["Year"] = btc_data["Date"].dt.year

In [ ]:
# Support and Resistance Levels
btc_data["Support"] = btc_data["Low"].rolling(window=20).min()  # Lowest low in the past 20 days
btc_data["Resistance"] = btc_data["High"].rolling(window=20).max()  # Highest high in the past 20 days

In [ ]:
# Calculate the percentage price change over the next 7 days (Target Variable for Regression)
btc_data["Pct_Change"] = ((btc_data["Price"].shift(-7) - btc_data["Price"]) / btc_data["Price"]) * 100

In [ ]:
# Drop the original "Vol. ('000)" column
btc_data.drop(columns=["Vol. ('000)"], inplace=True)

# Reset index
btc_data.reset_index(drop=True, inplace=True)

In [ ]:
btc_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4112 entries, 0 to 4111
Data columns (total 26 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           4112 non-null   datetime64[ns]
 1   Price          1129 non-null   float64       
 2   Open           4112 non-null   object        
 3   High           1116 non-null   float64       
 4   Low            1144 non-null   float64       
      4112 non-null   object        
 6   RSI            4093 non-null   float64       
 7   EMA            4099 non-null   float64       
 8   SMA            1074 non-null   float64       
 9   MACD           4087 non-null   float64       
 10  MACD_Signal    4079 non-null   float64       
 11  MACD_Hist      4079 non-null   float64       
 12  BB_Upper       1061 non-null   float64       
 13  BB_Middle      1061 non-null   float64       
 14  BB_Lower       1061 non-null   float64       
 15  BB_Percentage  1061 non-null   floa

In [ ]:
btc_data.head()

,Date,Price,Open,High,Low,Change %\r,RSI,EMA,SMA,MACD,...,Volume,OBV,Day_of_Week,Week_of_Year,Month,Quarter,Year,Support,Resistance,Pct_Change
0,2014-01-01,815.9,805.9,829.9,771.0,1.24%\r,NaN,NaN,NaN,NaN,...,10.76,10.76,2,1,1,1,2014,NaN,NaN,15.063120
1,2014-01-02,856.9,815.9,886.2,810.5,5.02%\r,NaN,NaN,NaN,NaN,...,12.81,23.57,3,1,1,1,2014,NaN,NaN,9.347649
2,2014-01-03,884.3,856.9,888.2,839.4,3.19%\r,NaN,NaN,NaN,NaN,...,9.71,33.28,4,1,1,1,2014,NaN,NaN,8.311659
3,2014-01-04,924.7,884.3,932.2,848.3,4.57%\r,NaN,NaN,NaN,NaN,...,14.24,47.52,5,1,1,1,2014,NaN,NaN,NaN
4,2014-01-05,NaN,924.7,NaN,911.4,9.74%\r,NaN,NaN,NaN,NaN,...,21.37,NaN,6,1,1,1,2014,NaN,NaN,NaN


In [ ]:
btc_data.tail()

,Date,Price,Open,High,Low,Change %\r,RSI,EMA,SMA,MACD,...,Volume,OBV,Day_of_Week,Week_of_Year,Month,Quarter,Year,Support,Resistance,Pct_Change
4107,2025-04-01,NaN,"82,548.60",NaN,NaN,3.17%\r,95.393727,964.420986,NaN,11.7338,...,66.72,NaN,1,14,4,2,2025,NaN,NaN,NaN
4108,2025-04-02,NaN,"85,166.00",NaN,NaN,-3.10%\r,95.393727,964.420986,NaN,11.7338,...,104.05,NaN,2,14,4,2,2025,NaN,NaN,NaN
4109,2025-04-03,NaN,"82,522.60",NaN,NaN,0.77%\r,95.393727,964.420986,NaN,11.7338,...,83.15,NaN,3,14,4,2,2025,NaN,NaN,NaN
4110,2025-04-04,NaN,"83,160.20",NaN,NaN,0.85%\r,95.393727,964.420986,NaN,11.7338,...,91.54,NaN,4,14,4,2,2025,NaN,NaN,NaN
4111,2025-04-05,NaN,"83,876.00",NaN,NaN,-0.38%\r,95.393727,964.420986,NaN,11.7338,...,32.71,NaN,5,14,4,2,2025,NaN,NaN,NaN
